In [9]:
import os

try:
    # Running as normal Python script inside src/
    this_file = os.path.abspath(__file__)
    src_root = os.path.dirname(this_file)                        # EMOTION-PRED/src
    project_root = os.path.dirname(src_root)                    # EMOTION-PRED/
except NameError:
    # Running inside Jupyter (likely src/notebooks or src/)
    cwd = os.getcwd()

    # If running inside src/notebooks → go up one level
    if cwd.endswith("notebooks"):
        src_root = os.path.abspath(os.path.join(cwd, ".."))
        project_root = os.path.dirname(src_root)
    else:
        # Running from project root directly
        project_root = cwd
        src_root = os.path.join(project_root, "src")

# Final unified paths
results_root = os.path.join(src_root, "results")
data_root = os.path.join(src_root, "data","MAMS-ACSA","raw","data_jsonl")
print(f"📂 Project root: {project_root}"
      f"\n📂 Source root: {src_root}"
      f"\n📂 Results root: {results_root}"
      f"\n📂 Data root: {data_root}")
# 3 — JSONL files
TRAIN_JSONL = os.path.join(data_root, "train.jsonl")
VAL_JSONL   = os.path.join(data_root, "val.jsonl")
TEST_JSONL  = os.path.join(data_root, "test.jsonl")
SAMPLE_JSONL = os.path.join(data_root, "sample.jsonl")
print("Using dataset directory:", data_root)



📂 Project root: /Users/hd/Desktop/EMOTION-PRED
📂 Source root: /Users/hd/Desktop/EMOTION-PRED/src
📂 Results root: /Users/hd/Desktop/EMOTION-PRED/src/results
📂 Data root: /Users/hd/Desktop/EMOTION-PRED/src/data/MAMS-ACSA/raw/data_jsonl
Using dataset directory: /Users/hd/Desktop/EMOTION-PRED/src/data/MAMS-ACSA/raw/data_jsonl


In [10]:
# # ============================================================
# # VALIDATION SCRIPT FOR 300 GOLD ANNOTATIONS
# # Evaluates:
# #  - Emotion Macro-F1 / Micro-F1
# #  - Aspect Macro-F1
# #  - Polarity Macro-F1
# #  - Full ABSA triple accuracy
# # ============================================================

# import json
# import pandas as pd
# from sklearn.metrics import f1_score, classification_report, precision_recall_fscore_support

# # ------------------------------------------------------------
# # PATHS - UPDATE ONLY THESE IF NEEDED
# # ------------------------------------------------------------
# GOLD_PATH = os.path.join(data_root, "sample_06_12_2025_6pm_annotated.jsonl")

# # Option A → Emotion-only model output
# PRED_PATH = "output/gemini_annotated.jsonl"

# # Option B → Full aspect+polarity+emotion prediction
# # PRED_PATH = "output/gemini_full_absa.jsonl"

# # ------------------------------------------------------------
# # LOAD FILES
# # ------------------------------------------------------------
# def load_jsonl(path):
#     rows = []
#     with open(path, "r", encoding="utf-8") as f:
#         for line in f:
#             rows.append(json.loads(line))
#     return rows

# gold = load_jsonl(GOLD_PATH)
# pred = load_jsonl(PRED_PATH)

# assert len(gold) == len(pred), f"Row mismatch: gold={len(gold)}, pred={len(pred)}"
# print(f"Loaded {len(gold)} rows ✓")

# # ------------------------------------------------------------
# # EXPLODE INTO FLAT FORM
# # ------------------------------------------------------------
# def explode(data):
#     rows = []
#     for i, row in enumerate(data):
#         for item in row["output"]:
#             rows.append({
#                 "id": i,
#                 "aspect": item.get("aspect"),
#                 "polarity": item.get("polarity"),
#                 "emotion": item.get("emotion")
#             })
#     return pd.DataFrame(rows)

# gold_df = explode(gold)
# pred_df = explode(pred)

# print("Gold rows:", len(gold_df))
# print("Pred rows:", len(pred_df))

# # ------------------------------------------------------------
# # MERGE GOLD + PRED ON (ID, ASPECT, POLARITY)
# # ------------------------------------------------------------
# merged = gold_df.merge(
#     pred_df,
#     on=["id", "aspect", "polarity"],
#     suffixes=("_gold", "_pred"),
#     how="outer"
# )

# print("Merged rows:", len(merged))

# # Drop rows where emotion missing in gold or pred
# valid = merged.dropna(subset=["emotion_gold", "emotion_pred"])

# # ------------------------------------------------------------
# # EMOTION F1
# # ------------------------------------------------------------
# y_true = valid["emotion_gold"]
# y_pred = valid["emotion_pred"]

# print("\n==============================")
# print("EMOTION SCORES")
# print("==============================")
# print("Emotion Macro-F1:", f1_score(y_true, y_pred, average='macro'))
# print("Emotion Micro-F1:", f1_score(y_true, y_pred, average='micro'))

# print("\nClassification report:")
# print(classification_report(y_true, y_pred))

# # ------------------------------------------------------------
# # ASPECT F1 (if full ABSA predictions contain aspects)
# # ------------------------------------------------------------
# print("\n==============================")
# print("ASPECT SCORES")
# print("==============================")

# aspect_true = gold_df["aspect"]
# aspect_pred = pred_df["aspect"]

# aspect_f1 = f1_score(aspect_true, aspect_pred, average="macro")
# print("Aspect Macro-F1:", aspect_f1)

# # ------------------------------------------------------------
# # POLARITY F1
# # ------------------------------------------------------------
# print("\n==============================")
# print("POLARITY SCORES")
# print("==============================")

# pol_true = gold_df["polarity"]
# pol_pred = pred_df["polarity"]

# pol_f1 = f1_score(pol_true, pol_pred, average="macro")
# print("Polarity Macro-F1:", pol_f1)

# # ------------------------------------------------------------
# # FULL ABSA TRIPLE ACCURACY
# # aspect + polarity + emotion must match exactly
# # ------------------------------------------------------------
# print("\n==============================")
# print("FULL ABSA TRIPLE ACCURACY")
# print("==============================")

# merged["triple_gold"] = list(
#     zip(merged["aspect"], merged["polarity"], merged["emotion_gold"])
# )
# merged["triple_pred"] = list(
#     zip(merged["aspect"], merged["polarity"], merged["emotion_pred"])
# )

# merged["correct_triple"] = merged["triple_gold"] == merged["triple_pred"]
# triple_accuracy = merged["correct_triple"].mean()

# print("Full ABSA Accuracy:", triple_accuracy)

# # ------------------------------------------------------------
# # SUMMARY TABLE
# # ------------------------------------------------------------
# print("\n==============================")
# print("SUMMARY")
# print("==============================")

# print(f"""
# Emotion Macro-F1      : {f1_score(y_true, y_pred, average='macro'):.4f}
# Emotion Micro-F1      : {f1_score(y_true, y_pred, average='micro'):.4f}
# Aspect Macro-F1       : {aspect_f1:.4f}
# Polarity Macro-F1     : {pol_f1:.4f}
# Full ABSA Accuracy    : {triple_accuracy:.4f}
# """)

In [ ]:
import json
import os
import pandas as pd
from sklearn.metrics import classification_report, f1_score
from collections import Counter

# -----------------------------
# Paths
# -----------------------------
GOLD_PATH = os.path.join(data_root, "sample_06_12_2025_6pm_annotated.jsonl")
PRED_DIR  = os.path.join(src_root,"notebooks","output","gemini_annotated_aspect_polarity_emotions.jsonl")  # folder with Gemini / GPT / LLaMA predictions

# -----------------------------
# Load JSONL
# -----------------------------
def load_jsonl(path):
    return [json.loads(line) for line in open(path, "r", encoding="utf-8")]

gold = load_jsonl(GOLD_PATH)

# Index gold by text for alignment
gold_map = {row["input"]: row["output"] for row in gold}

# -----------------------------
# Convert triple list into flat lists for scoring
# -----------------------------
def flatten_triples(triples):
    """
    Each review creates N triples → but classification requires aligned lists.
    We treat each triple independently.
    """
    aspects = []
    polarities = []
    emotions = []
    triples_clean = []

    for t in triples:
        asp = t["aspect"]
        pol = t["polarity"]
        emo = t["emotion"]

        aspects.append(asp)
        polarities.append(pol)
        emotions.append(emo)
        triples_clean.append((asp, pol, emo))

    return aspects, polarities, emotions, triples_clean


# -----------------------------
# Evaluate a single prediction file
# -----------------------------
def evaluate_prediction_file(pred_path):
    pred = load_jsonl(pred_path)

    # Align using input text
    pred_map = {row["input"]: row["output"] for row in pred}

    gold_aspects = []
    gold_pols    = []
    gold_emots   = []
    gold_triples = []

    pred_aspects = []
    pred_pols    = []
    pred_emots   = []
    pred_triples = []

    # Iterate through all rows
    for text, gold_list in gold_map.items():
        pred_list = pred_map.get(text, [])

        ga, gp, ge, gt = flatten_triples(gold_list)
        pa, pp, pe, pt = flatten_triples(pred_list)

        gold_aspects.extend(ga)
        gold_pols.extend(gp)
        gold_emots.extend(ge)
        gold_triples.extend(gt)

        pred_aspects.extend(pa)
        pred_pols.extend(pp)
        pred_emots.extend(pe)
        pred_triples.extend(pt)

    # -----------------------------
    # Aspect F1
    # -----------------------------
    aspect_f1 = f1_score(gold_aspects, pred_aspects, average="macro", zero_division=0)

    # -----------------------------
    # Polarity F1
    # -----------------------------
    polarity_f1 = f1_score(gold_pols, pred_pols, average="macro", zero_division=0)

    # -----------------------------
    # Emotion F1 (macro/micro)
    # -----------------------------
    emo_macro = f1_score(gold_emots, pred_emots, average="macro", zero_division=0)
    emo_micro = f1_score(gold_emots, pred_emots, average="micro", zero_division=0)

    # -----------------------------
    # FULL ABSA Exact Triple Match
    # -----------------------------
    gold_counter = Counter(gold_triples)
    pred_counter = Counter(pred_triples)

    match = sum((gold_counter & pred_counter).values())
    total = len(gold_triples)
    exact_acc = match / total if total > 0 else 0

    # Return structured scores
    return {
        "file": os.path.basename(pred_path),
        "Aspect F1": aspect_f1,
        "Polarity F1": polarity_f1,
        "Emotion Macro-F1": emo_macro,
        "Emotion Micro-F1": emo_micro,
        "Full ABSA Accuracy": exact_acc
    }


# -----------------------------
# Evaluate ALL prediction files
# -----------------------------
results = []

for fname in os.listdir(PRED_DIR):
    if fname.endswith(".jsonl") and "annotated" in fname:
        pred_path = os.path.join(PRED_DIR, fname)
        print("Evaluating:", pred_path)
        scores = evaluate_prediction_file(pred_path)
        results.append(scores)

df = pd.DataFrame(results)
print(df)

# Save CSV summary
df.to_csv("absa_model_comparison.csv", index=False)
print("\nSaved → absa_model_comparison.csv")

NotADirectoryError: [Errno 20] Not a directory: 'output/gemini_annotated_aspect_polarity_emotions.jsonl'